In [ ]:
import torch
from torchvision import datasets, transforms
from tqdm import tqdm

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

print(trainloader.dataset.data.shape) # 60000 * 28  28

dataiter = iter(trainloader)
images, labels = next(dataiter)
print(images.shape[-1])
images = torch.cat((images, images, images), 1)
print(images.shape)
# print("images shape:", images.shape, "labels shape:", labels.shape)


In [ ]:
import torch
import torchvision.models as models

model = models.resnet18()
print(model)

# Input shape
input_shape = (3, 224, 224)
x = torch.randn(1, *input_shape)
print(f"Input shape: {x.shape}")

# Summary
from torchsummary import summary
summary(model, input_shape)


In [2]:
from domain import Network
from dataset import mnist_train_loader, mnistm_train_loader
import torch
import torch.nn as nn

train_source, train_target = mnist_train_loader, mnistm_train_loader

net = Network(train_source, train_target)
# net._train_source(epochs=20)
# net._train_dann(epochs=100)
net._test_source()

# print(torch.cuda.device_count())
if torch.cuda.device_count() > 1:
    layout = list(net.model.module.children())
else:
    layout = list(net.model.children())

model = layout[:-1]
model = nn.Sequential(*model)

model.eval()  
with torch.no_grad():
    dummpy_input = torch.randn(1, 3, 28, 28).to(net.device)
    output = model(dummpy_input)
    print(output.shape)


# print(net.E)
# print(nn.Sequential(*list(net.model.children())))



>> Only-Source Domain Test Start


RuntimeError: module must have its parameters and buffers on device cuda:0 (device_ids[0]) but found one of them on device: cpu

In [ ]:
print(net.model)
"""
ResNet18은 일반적으로 224x224x3 크기의 이미지를 입력을 처리하도록 설계되어있음
하지만, 우리의 데이터셋은 28x28x3 크기의 이미지를 가짐
따라서, ResNet18의 입력 크기를 조정해야 함

ResNet18의 입력 크기를 조정하는 방법은 두 가지가 있음
1. 입력 크기를 224x224x3으로 조정
2. ResNet18의 첫 번째 계층을 3x3 커널, 2 스트라이드, 1 패딩으로 변경

입력: 28x28x3

첫 번째 Conv2d (7x7 커널, 2 스트라이드, 3 패딩): 
(28+2x3-7)/2+1=15
(28+2x3-7)/2+1=15 -> 15x15x64

첫 번째 MaxPool2d (3x3 커널, 2 스트라이드, 1 패딩): 
(15+2x1-3)/2+1=8
(15+2x1-3)/2+1=8 -> 8x8x64

이후의 계층들은 ResNet의 기본 블록
이 블록들은 입력 특징 맵의 크기를 변경하지 않지만, 채널 수는 블록의 각 반복마다 두 배씩 증가할 수 있음.

Layer1 (ResNet 블록 x2): 8x8x64
Layer2 (ResNet 블록 x2, 첫 번째 블록에서 스트라이드=2로 다운샘플링): 4x4x128
Layer3 (ResNet 블록 x2, 첫 번째 블록에서 스트라이드=2로 다운샘플링): 2x2x256
Layer4 (ResNet 블록 x2, 첫 번째 블록에서 스트라이드=2로 다운샘플링): 1x1x512
"""

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define the batch size
batch_size = 64

# Define the transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Define the model
model = nn.Sequential(nn.Linear(784, 128),
                    nn.ReLU(),
                    nn.Linear(128, 64),
                    nn.ReLU(),
                    nn.Linear(64, 10),
                    nn.LogSoftmax(dim=1))

# Define the loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

# Train the model
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
        
        # Clear the gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(images)
        loss = criterion(output, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")


>> Only-Source Domain Training Start


  0%|          | 0/1719 [00:00<?, ?it/s]


TypeError: Tensor.backward() got an unexpected keyword argument 'distance'